# **Basic MoSDeF Workflow**

## Introduction for Lyon lab members
There are two ways to use this Jupyter Notebook:

1. Installing Anaconda (anaconda.org) or Miniconda in your computer and running locally this notebook.

2. Accesing this notebook through the <a href="https://github.com/lyon-group/Intro_to_MoSDeF">Lyon Group GitHub repository</a> and opening it either in Google Collab or GitHub workspaces (Google Collab is recommended if using Chapman email).


## Important considerations
There are two types of output in these Colab notebooks that can be a little trick

1. If the output is very long, for example from the mamba command in the second cell, scrolling past the output can feel onerous. In this case, scrolling up and down in the narrow grey area between the sidebar menu and the cells can help you navigate.

2. If the output is a visualization of a molecule or simulation configuration, scrolling up or down will zoom in or out if the cursor is over the visualization. In these cases, take some care to scroll outside of the visualization.

3. To run a cell, either click the run button (right facing triangle) or hit `shift + enter`

In [1]:
import warnings
#warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Import Libraries to be used
import os
import mbuild as mb
import gmso

/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefield.py:33: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import iter_entry_points, resource_filename
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/pkg_resources/__init__.py:3144: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/c

In [15]:
# Load structure from recipes (delivered with mBuild), SMILES string, or external pdb/mol2 file
from mbuild.lib.molecules.water import WaterTIP3P
water = WaterTIP3P()
water.name = 'HOH'
water.save("water_tip3p.pdb", overwrite=True)
#water.energy_minimize(forcefield='oplsaa', steps=10**4)

pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipam.name = 'PNP'
pnipam.save("nipam.pdb", overwrite=True)
pnipam.visualize()



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
# Load structure from recipes (delivered with mBuild), SMILES string, or external pdb/mol2 file
pnipam = mb.load("pnipam_pubchem.pdb")
#pnipam.name = 'PNP'
#pnipam.save("nipam.pdb", overwrite=True)
pnipam.visualize()



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#Create a simulation box using Packmol for a binary system
sim_box = mb.fill_box(compound= [water, pnipam],
                      density=1022.3,
                      compound_ratio=[0.4, 0.6],
                      box=[5.0, 5.0, 5.0])
sim_box.visualize()

In [16]:
#Create a simulation box using Packmol for a single-component system
### Using density
#sim_box = mb.fill_box(pnipam, density=1022.3, box=[5.0, 5.0, 5.0])
### Using number of molecules
sim_box = mb.fill_box(water, box=[5,5,5], n_compounds=680)
sim_box.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [17]:
#Visualizing the 
"""Visualization utilities"""
print(sim_box.print_hierarchy(show_tree=False))  # print_hierarchy() in normal colab
#Optionally, you can save your topology to .pdb, .mol2, .xyz, etc.
sim_box.save("sim_box.pdb", overwrite=True)
sim_box.visualize()

Compound, 2040 particles, 1360 bonds, 680 children
└── [HOH x 680], 3 particles, 2 bonds, 3 children
    ├── [HW1 x 1], 1 particles, 1 bonds, 0 children
    ├── [HW2 x 1], 1 particles, 1 bonds, 0 children
    └── [OW x 1], 1 particles, 2 bonds, 0 children



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
#Loading forcefields
warnings.filterwarnings("ignore", category=DeprecationWarning) 

tip3p = gmso.ForceField("/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefields/xml/tip3p.xml")
oplsaa = gmso.ForceField("OPLSAA")
gaff = gmso.ForceField("/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/foyer/forcefields/xml/gaff.xml")
#gaff = gmso.ForceField("GAFF")
#gaff

/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/gmso/core/atom_type.py:236: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/Users/cardenasvasquez/anaconda3/envs/mosdef/lib/python3.10/site-packages/gmso/core/atom_type.py:248: UserWarning: Charges are assumed to be in units of elementary charge
  warnings.warn(UNIT_WARNING_STRING.format("Charges", "elementary charge"))


In [18]:
#Adding forcefield to pre-loaded molecules or simulation boxes
from gmso.parameterization import apply

#topology
pnipam_top = sim_box.to_gmso()

#water_ptop = apply(water_top, tip3p, identify_connections=True)
pnipam_ptop = apply(pnipam_top, gaff, identify_connections=True)

In [11]:
display(water_ptop.sites[0].atom_type.expression)
print(f"{water_ptop.sites[0].atom_type.expression}")

NameError: name 'water_ptop' is not defined

In [12]:
display(pnipam_ptop.sites[0].atom_type.expression)
print(f"{pnipam_ptop.sites[0].atom_type.expression}")

4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)

4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)


In [19]:
"""Utility to output system as Dataframe"""
#water_ptop.to_dataframe(site_attrs=["atom_type.parameters"])
pnipam_ptop.to_dataframe(site_attrs=["atom_type.parameters"])

,atom_types,names,atom_type.parameters
0,oh,OW,"{'epsilon': 0.8803136 kJ/mol, 'sigma': 0.30664..."
1,ho,HW1,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"
2,ho,HW2,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"
3,oh,OW,"{'epsilon': 0.8803136 kJ/mol, 'sigma': 0.30664..."
4,ho,HW1,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"
...,...,...,...
2035,ho,HW1,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"
2036,ho,HW2,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"
2037,oh,OW,"{'epsilon': 0.8803136 kJ/mol, 'sigma': 0.30664..."
2038,ho,HW1,"{'epsilon': 0.0 kJ/mol, 'sigma': 0.0 nm}"


In [20]:
pnipam_ptop.save("pnipam_box.lammps", overwrite=True)
!cat pnipam_box.lammps

Topology written by cardenasvasquez at 2024-10-21 10:39:28.115294 using the GMSO LAMMPS Writer


2040 atoms
1360 bonds
680 angles
0 dihedrals
0 impropers

2 atom types
1 bond types
1 angle types

0.000000 50.000000 xlo xhi
0.000000 50.000000 ylo yhi
0.000000 50.000000 zlo zhi
0.000000 0.000000 0.000000 xy xz yz

Masses
#	mass (amu)
1	1.008	# ho
2	16.000	# oh

Pair Coeffs # 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)
#	epsilon (kcal/mol)	sigma (Å)
1	0.00000		0.00000		# ho
2	0.21040		3.06647		# oh

Bond Coeffs #LAMMPSHarmonicBondPotential
#	k (kcal/(mol*Å**2))	r_eq (Å)
1	371.400000	0.973000		# ho	oh

Angle Coeffs #LAMMPSHarmonicAnglePotential
#	k (kcal/(mol*rad**2))	theta_eq (degrees)
1	41.600000	106.490000	#ho         	oh         	ho         

Atoms #full

1	1	2	0.000000	21.58338	3.954715	39.18531
2	1	1	0.000000	21.00180	4.279582	39.87267
3	1	1	0.000000	20.99722	3.546633	38.54803
4	2	2	0.000000	28.24718	18.82711	3.617320
5	2	1	0.000000	28.20780	19.19234	4.501225
6	2	1	0.000000	27.38459

In [ ]:
gmso.formats.write_lammpsdata(pnipam_top, "out.lammps", atom_style='molecular', unit_style='real', strict_potentials=False, strict_units=True, lj_cfactorsDict=None)



In [ ]:
#pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipam.visualize()
#C=CC(=O)O

In [ ]:
pnipam.name = 'PNP'
pnipam.energy_minimize(forcefield='oplsaa', steps=10**4)

In [ ]:
aac = mb.load("C=CC(=O)O", smiles=True)
aac.visualize()

In [ ]:
#pnipam = mb.load("CC(C)NC(=O)C=C", smiles=True)
pnipamco = mb.load("CC(C)NC(=O)CC(C)(C(=O)O)", smiles=True)
pnipamco.visualize()
#C=CC(=O)O

In [ ]:
#pnipam_co_aac = mb.load("CC(C(=O)NC(C)C)C(C(=O)O)(C)", smiles=True)
pnipam_co_aac = mb.load("CC(C(=O)NC(C)C)C(C(=O)O)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)NC(C)C)C(C(=O)O)(C)C", smiles=True)
pnipam_co_aac.visualize()